<a href="https://colab.research.google.com/github/patriciaapenat/TFM/blob/main/Notebooks/Data_Frame_covid_daily_oms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#! git clone https://github.com/patriciaapenat/TFM.git

### **Fuente de Datos - Ficheros WHO OMS**
**https://covid19.who.int/data**<br>
**https://ourworldindata.org/coronavirus#coronavirus-country-profiles**<br>


#### Descarga de datos

**1.- Casos diarios y muertes por fecha notificados a la OMS:** https://covid19.who.int/WHO-COVID-19-global-data.csv<br>
**2.- Últimos recuentos informados de casos y muertes:** https://covid19.who.int/WHO-COVID-19-global-table-data.csv<br>
**3.- Datos de vacunación:** https://covid19.who.int/who-data/vaccination-data.csv<br>
**4.- Metadatos de vacunación:**https://covid19.who.int/who-data/vaccination-metadata.csv<br>


**🧾Informacion del Dataset**<br>
*  Los usuarios deben tener en cuenta que, además de capturar nuevos casos y muertes notificadas en un día determinado, las actualizaciones se realizan retrospectivamente para corregir los recuentos de días anteriores según sea necesario en función de la información recibida posteriormente. Consulte "Datos agregados diarios de recuento de casos y muertes" más arriba para obtener más detalles sobre el cálculo de nuevos casos/muertes.

In [1]:
import pandas as pd
import numpy as np

#!pip install mysql-connector-python
#!pip install sqlalchemy
#!pip install PyMySql
#!pip install ipython-sql

In [2]:
#pip install pycountry==20.7.3
#!pip install pycountry-convert==0.7.2

# Acceso a los datasets de la OMS:

Exloración e Información del DataFrame **df_covid_daily**

In [3]:
df_covid_daily = pd.read_csv('https://covid19.who.int/WHO-COVID-19-global-data.csv')
df_covid_daily.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-03,AF,Afghanistan,EMRO,0,0,0,0
1,2020-01-04,AF,Afghanistan,EMRO,0,0,0,0
2,2020-01-05,AF,Afghanistan,EMRO,0,0,0,0
3,2020-01-06,AF,Afghanistan,EMRO,0,0,0,0
4,2020-01-07,AF,Afghanistan,EMRO,0,0,0,0


In [4]:
df_covid_daily.shape

(328245, 8)

In [5]:
df_covid_daily.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328245 entries, 0 to 328244
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Date_reported      328245 non-null  object
 1   Country_code       326860 non-null  object
 2   Country            328245 non-null  object
 3   WHO_region         328245 non-null  object
 4   New_cases          328245 non-null  int64 
 5   Cumulative_cases   328245 non-null  int64 
 6   New_deaths         328245 non-null  int64 
 7   Cumulative_deaths  328245 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 20.0+ MB


Transformamos **Date_reported** a formato datetime con formato **dd/mm/yy**

In [6]:
df_covid_daily['Date_reported']= pd.to_datetime(df_covid_daily['Date_reported']).dt.strftime('%d-%m-%Y')

Verificamos que se han modificado el tipo de datos a formato fecha **dd/mm/yy**

In [7]:
df_covid_daily.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,03-01-2020,AF,Afghanistan,EMRO,0,0,0,0
1,04-01-2020,AF,Afghanistan,EMRO,0,0,0,0
2,05-01-2020,AF,Afghanistan,EMRO,0,0,0,0
3,06-01-2020,AF,Afghanistan,EMRO,0,0,0,0
4,07-01-2020,AF,Afghanistan,EMRO,0,0,0,0


Observamos que no hay **ningún valor duplicado**

In [8]:
df_covid_daily.duplicated().values.any()

False

Observamos que hay **1379 valores nulos**

In [9]:
df_covid_daily.isnull().values.sum()

1385

Identificamos en que **Variable** y la **suma de valores nulos**

In [10]:
df_covid_daily.isnull().sum()

Date_reported           0
Country_code         1385
Country                 0
WHO_region              0
New_cases               0
Cumulative_cases        0
New_deaths              0
Cumulative_deaths       0
dtype: int64

Observamos que el Country Namibia tiene código Null debido a que su código ISO es **NM**

In [11]:
df_covid_daily[df_covid_daily['Country_code'].isnull()]

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
198055,03-01-2020,NaN,Namibia,AFRO,0,0,0,0
198056,04-01-2020,NaN,Namibia,AFRO,0,0,0,0
198057,05-01-2020,NaN,Namibia,AFRO,0,0,0,0
198058,06-01-2020,NaN,Namibia,AFRO,0,0,0,0
198059,07-01-2020,NaN,Namibia,AFRO,0,0,0,0
...,...,...,...,...,...,...,...,...
199435,14-10-2023,NaN,Namibia,AFRO,0,172121,0,4100
199436,15-10-2023,NaN,Namibia,AFRO,0,172121,0,4100
199437,16-10-2023,NaN,Namibia,AFRO,0,172121,0,4100
199438,17-10-2023,NaN,Namibia,AFRO,0,172121,0,4100


Sustituimos los valores Null **NaM** de Namibia por el Código ISO2 **NM**

In [12]:
df_covid_daily=df_covid_daily.fillna('NM')
df_covid_daily.head()

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,03-01-2020,AF,Afghanistan,EMRO,0,0,0,0
1,04-01-2020,AF,Afghanistan,EMRO,0,0,0,0
2,05-01-2020,AF,Afghanistan,EMRO,0,0,0,0
3,06-01-2020,AF,Afghanistan,EMRO,0,0,0,0
4,07-01-2020,AF,Afghanistan,EMRO,0,0,0,0


Verificamos que se ha cambiado correctamente el código ISO2 de Namibia a **NM**

In [13]:
df_covid_daily[df_covid_daily['Country']=="Namibia"]

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
198055,03-01-2020,NM,Namibia,AFRO,0,0,0,0
198056,04-01-2020,NM,Namibia,AFRO,0,0,0,0
198057,05-01-2020,NM,Namibia,AFRO,0,0,0,0
198058,06-01-2020,NM,Namibia,AFRO,0,0,0,0
198059,07-01-2020,NM,Namibia,AFRO,0,0,0,0
...,...,...,...,...,...,...,...,...
199435,14-10-2023,NM,Namibia,AFRO,0,172121,0,4100
199436,15-10-2023,NM,Namibia,AFRO,0,172121,0,4100
199437,16-10-2023,NM,Namibia,AFRO,0,172121,0,4100
199438,17-10-2023,NM,Namibia,AFRO,0,172121,0,4100


Ya no tenemos vamores Nulos en ninguna variable del Dataframe

In [14]:
df_covid_daily.isnull().sum()

Date_reported        0
Country_code         0
Country              0
WHO_region           0
New_cases            0
Cumulative_cases     0
New_deaths           0
Cumulative_deaths    0
dtype: int64

Cargo el fichero **paises\_def** **que está en el Repositorio de GitHub**

In [15]:
df_paises = pd.read_csv('C:/Users/joschava/Dropbox/TFM COVID/Entregables/2023.10 Entrega Final/Tablas Soporte/paises_def.csv')
df_paises.head()

,PAIS_ISO3,PAIS_ISO2,PAIS_NOM,COD_CONTINENTE,CONTINENTE,OMS_REGION,DESC_OMS_REGION,PAIS_NOM_2,POBLACION,LONGITUD,LATITUD,GDP_PER_CAPITA,USO_INDIVIDUAL_INTERNET,USO_DE_AGUA_POTABLE,USO_DE_SERVICIOS_BASICOS_SANEAMIENTO,DENSIDAD_DE_POBLACION_X_KM2,POBLACION_RURAL,POBLACION_URBANA,TEMPERATURA_PROMEDIO,ALTURA_PROMEDIO
0,AGO,AO,Angola,AF,Africa,AFRO,Africa,Angola,35588987.0,17.873887,-11.202692,1502.950754,32.550147,57.167738,51.660329,26.813577,33.175,66.825,22.508965,1112.0
1,BDI,BI,Burundi,AF,Africa,AFRO,Africa,Burundi,12889576.0,29.918886,-3.373056,216.827417,3.787954,62.207122,45.727511,475.865537,86.292,13.708,20.816609,1504.0
2,BEN,BJ,Benin,AF,Africa,AFRO,Africa,Benin,13352864.0,2.315834,9.307690,1237.949295,28.229154,65.414123,16.962884,112.124184,51.585,48.415,27.937785,273.0
3,BFA,BF,Burkina Faso,AF,Africa,AFRO,Africa,Burkina Faso,22673762.0,-1.561593,12.238333,833.244336,13.937972,47.214854,21.664373,78.664569,69.393,30.607,28.909989,297.0
4,BWA,BW,Botswana,AF,Africa,AFRO,Africa,Botswana,2630296.0,24.684866,-22.328474,5875.070606,72.748020,92.213563,80.026243,4.493148,29.123,70.877,22.505484,1013.0


Realizo la unión entre ambos dataframes por la variable **ISO2**

In [16]:
df_covid_daily_new = pd.merge(left=df_covid_daily, right=df_paises,
                      how='left', left_on='Country_code', right_on='PAIS_ISO2')

In [17]:
df_covid_daily_new.isnull().sum()

Date_reported                                0
Country_code                                 0
Country                                      0
WHO_region                                   0
New_cases                                    0
Cumulative_cases                             0
New_deaths                                   0
Cumulative_deaths                            0
PAIS_ISO3                                 6925
PAIS_ISO2                                 6925
PAIS_NOM                                  6925
COD_CONTINENTE                           60940
CONTINENTE                                6925
OMS_REGION                                6925
DESC_OMS_REGION                           6925
PAIS_NOM_2                                6925
POBLACION                                33240
LONGITUD                                 15235
LATITUD                                  15235
GDP_PER_CAPITA                           42935
USO_INDIVIDUAL_INTERNET                  74790
USO_DE_AGUA_P

Identificamos 6895 valores nulos pero corresponden a códigos ISO2 especiales y los sustituiré por el valor **Z9**

In [18]:
df_covid_daily_new[df_covid_daily_new['PAIS_ISO3'].isnull()]

,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,PAIS_ISO3,PAIS_ISO2,...,LATITUD,GDP_PER_CAPITA,USO_INDIVIDUAL_INTERNET,USO_DE_AGUA_POTABLE,USO_DE_SERVICIOS_BASICOS_SANEAMIENTO,DENSIDAD_DE_POBLACION_X_KM2,POBLACION_RURAL,POBLACION_URBANA,TEMPERATURA_PROMEDIO,ALTURA_PROMEDIO
34625,03-01-2020,XA,Bonaire,AMRO,0,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34626,04-01-2020,XA,Bonaire,AMRO,0,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34627,05-01-2020,XA,Bonaire,AMRO,0,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34628,06-01-2020,XA,Bonaire,AMRO,0,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34629,07-01-2020,XA,Bonaire,AMRO,0,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270070,14-10-2023,XB,Sint Eustatius,AMRO,0,1221,0,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270071,15-10-2023,XB,Sint Eustatius,AMRO,0,1221,0,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270072,16-10-2023,XB,Sint Eustatius,AMRO,0,1221,0,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
270073,17-10-2023,XB,Sint Eustatius,AMRO,0,1221,0,6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Codigos especiales **iso2** que no aparecen en listado de paises_region.csv

https://es.wikipedia.org/wiki/ISO_3166-1_alfa-2

El código **XA** está siendo utilizado por Suiza, como código de país por las Islas Canarias, a pesar de que IC ya está reservado con dicho propósito.23<br>
El código **XI** está siendo utilizado por el Gobierno del Reino Unido, como el prefijo de código de país del número EORI de Irlanda del Norte.24<br>
El código **XK** está siendo utilizado por la Comisión Europea,25​ el FMI, la SWIFT,26​ el CLDR y otras organizaciones como código de país provisional para Kosovo.27<br>
El código **XN** está siendo utilizado por la Organización Mundial de la Propiedad Intelectual como indicador para el Instituto Nórdico de Patentes, una organización internacional a la que pertenecen Dinamarca, Islandia, Noruega y Suecia.28<br>
El código **XU** está siendo utilizado por la Organización Mundial de la Propiedad Intelectual como indicador para la Unión Internacional para la Protección de las Obtenciones Vegetales<br>
El código **XV** está siendo utilizado por la Organización Mundial de la Propiedad Intelectual como indicador para el Instituto Visegrad de Patentes<br>
El código **XX** está siendo utilizado por la Organización Mundial de la Propiedad Intelectual como un indicador para estados desconocidos, otras entidades u organizaciones<br>

Elimino columnas del dataframe

In [19]:
df_covid_daily_new.drop(['Country_code','Country','COD_CONTINENTE','CONTINENTE','OMS_REGION','DESC_OMS_REGION','PAIS_NOM_2','POBLACION','LONGITUD','LATITUD','GDP_PER_CAPITA','USO_INDIVIDUAL_INTERNET','USO_DE_AGUA_POTABLE','USO_DE_SERVICIOS_BASICOS_SANEAMIENTO','DENSIDAD_DE_POBLACION_X_KM2','POBLACION_RURAL','POBLACION_URBANA','TEMPERATURA_PROMEDIO','ALTURA_PROMEDIO'],axis=1,inplace=True)

In [20]:
df_covid_daily_new[df_covid_daily_new['PAIS_ISO3'].isnull()]

,Date_reported,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,PAIS_ISO3,PAIS_ISO2,PAIS_NOM
34625,03-01-2020,AMRO,0,0,0,0,NaN,NaN,NaN
34626,04-01-2020,AMRO,0,0,0,0,NaN,NaN,NaN
34627,05-01-2020,AMRO,0,0,0,0,NaN,NaN,NaN
34628,06-01-2020,AMRO,0,0,0,0,NaN,NaN,NaN
34629,07-01-2020,AMRO,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
270070,14-10-2023,AMRO,0,1221,0,6,NaN,NaN,NaN
270071,15-10-2023,AMRO,0,1221,0,6,NaN,NaN,NaN
270072,16-10-2023,AMRO,0,1221,0,6,NaN,NaN,NaN
270073,17-10-2023,AMRO,0,1221,0,6,NaN,NaN,NaN


Sustituir valores nulos por **Z9****

In [21]:
df_covid_daily_new['PAIS_ISO3'].fillna('Z99', inplace=True)
df_covid_daily_new['PAIS_ISO2'].fillna('Z9', inplace=True)
df_covid_daily_new['PAIS_NOM'].fillna('Otros', inplace=True)

Hacemos una última revisión. Observamos que hay 345 duplicados que son los registros que equivalen a **Z9**

In [22]:
df_covid_daily_new[df_covid_daily_new.duplicated()]

,Date_reported,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths,PAIS_ISO3,PAIS_ISO2,PAIS_NOM
245145,03-01-2020,AMRO,0,0,0,0,Z99,Z9,Otros
245146,04-01-2020,AMRO,0,0,0,0,Z99,Z9,Otros
245147,05-01-2020,AMRO,0,0,0,0,Z99,Z9,Otros
245148,06-01-2020,AMRO,0,0,0,0,Z99,Z9,Otros
245149,07-01-2020,AMRO,0,0,0,0,Z99,Z9,Otros
...,...,...,...,...,...,...,...,...,...
268903,03-08-2020,AMRO,0,3,0,0,Z99,Z9,Otros
268942,11-09-2020,AMRO,0,10,0,0,Z99,Z9,Otros
268943,12-09-2020,AMRO,0,10,0,0,Z99,Z9,Otros
268944,13-09-2020,AMRO,0,10,0,0,Z99,Z9,Otros


Observamos que ya **no existen valores nulos**

In [23]:
df_covid_daily_new.isnull().sum()

Date_reported        0
WHO_region           0
New_cases            0
Cumulative_cases     0
New_deaths           0
Cumulative_deaths    0
PAIS_ISO3            0
PAIS_ISO2            0
PAIS_NOM             0
dtype: int64

In [24]:
df_covid_daily_new.columns

Index(['Date_reported', 'WHO_region', 'New_cases', 'Cumulative_cases',
       'New_deaths', 'Cumulative_deaths', 'PAIS_ISO3', 'PAIS_ISO2',
       'PAIS_NOM'],
      dtype='object')

In [25]:
df_covid_daily_new = df_covid_daily_new.reindex(columns =['PAIS_NOM','PAIS_ISO3','PAIS_ISO2','Date_reported','WHO_region','New_cases','Cumulative_cases','New_deaths','Cumulative_deaths'])

In [26]:
df_covid_daily_new

,PAIS_NOM,PAIS_ISO3,PAIS_ISO2,Date_reported,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,Afghanistan,AFG,AF,03-01-2020,EMRO,0,0,0,0
1,Afghanistan,AFG,AF,04-01-2020,EMRO,0,0,0,0
2,Afghanistan,AFG,AF,05-01-2020,EMRO,0,0,0,0
3,Afghanistan,AFG,AF,06-01-2020,EMRO,0,0,0,0
4,Afghanistan,AFG,AF,07-01-2020,EMRO,0,0,0,0
...,...,...,...,...,...,...,...,...,...
328240,Zimbabwe,ZWE,ZW,14-10-2023,AFRO,0,265808,0,5718
328241,Zimbabwe,ZWE,ZW,15-10-2023,AFRO,0,265808,0,5718
328242,Zimbabwe,ZWE,ZW,16-10-2023,AFRO,0,265808,0,5718
328243,Zimbabwe,ZWE,ZW,17-10-2023,AFRO,0,265808,0,5718


In [32]:
df_covid_daily_new.columns=['PAIS','PAIS_ISO3','PAIS_ISO2','FECHA_INFORMADA','OMS_REGION','CASOS_NUEVOS','CASOS_ACUM','MUERTES_NUEVAS','MUERTES_ACUM']

In [33]:
df_covid_daily_new

,PAIS,PAIS_ISO3,PAIS_ISO2,FECHA_INFORMADA,OMS_REGION,CASOS_NUEVOS,CASOS_ACUM,MUERTES_NUEVAS,MUERTES_ACUM
0,Afghanistan,AFG,AF,03-01-2020,EMRO,0,0,0,0
1,Afghanistan,AFG,AF,04-01-2020,EMRO,0,0,0,0
2,Afghanistan,AFG,AF,05-01-2020,EMRO,0,0,0,0
3,Afghanistan,AFG,AF,06-01-2020,EMRO,0,0,0,0
4,Afghanistan,AFG,AF,07-01-2020,EMRO,0,0,0,0
...,...,...,...,...,...,...,...,...,...
328240,Zimbabwe,ZWE,ZW,14-10-2023,AFRO,0,265808,0,5718
328241,Zimbabwe,ZWE,ZW,15-10-2023,AFRO,0,265808,0,5718
328242,Zimbabwe,ZWE,ZW,16-10-2023,AFRO,0,265808,0,5718
328243,Zimbabwe,ZWE,ZW,17-10-2023,AFRO,0,265808,0,5718


In [ ]:
#df_covid_daily_new.to_csv('/content/TFM/Ficheros_Depurados/df_covid_daily_new.csv',index=False)

# Establecemos **Conexión a BBDD y Carga de Ficheros** (Azure Data Studio)

In [34]:
import pyodbc
server = 'servidortfmcovid-v2.database.windows.net'
database = 'basedatostfmcovid-v2'
username = 'admintfmcovid_v2'
password = 'TFMcovid2023!_v2'
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

# Nombre de la tabla en la base de datos
nombre_de_tabla = 'dbo.COVID_DAILY'  # Reemplaza 'MiTabla' por el nombre de tu tabla

# Eliminar registros
cursor.execute(f"DELETE FROM {nombre_de_tabla}")

# Inserta los datos del DataFrame en la tabla
for index, row in df_covid_daily_new.iterrows():
    insert_query = f"INSERT INTO {nombre_de_tabla} ({', '.join(df_covid_daily_new.columns)}) VALUES ({', '.join(['?'] * len(df_covid_daily_new.columns))})"
    cursor.execute(insert_query, tuple(row))
    cnxn.commit()

# Cierra la conexión
cnxn.close()

Error: ('01000', '[01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionWrite (send()). (10054) (SQLEndTran); [01000] [Microsoft][ODBC SQL Server Driver][DBNETLIB]General network error. Check your network documentation. (11)')